In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Initialize Variables
model_name = "AI-Sweden-Models/gpt-sw3-126m-instruct"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
prompt = "Träd är fina för att"

# Initialize Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
model.to(device)

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs=input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.6,
    top_p=1,
)[0]

generated_text = tokenizer.decode(generated_token_ids)

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs=input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.6,
    top_p=1,
)[0]

generator = pipeline('text-generation', tokenizer=tokenizer, model=model, device=device)
generated = generator(prompt, max_new_tokens=100, do_sample=True, temperature=0.6, top_p=1)[0]["generated_text"]


In [ ]:
print(generated)

In [ ]:
from transformers import StoppingCriteriaList, StoppingCriteria

prompt = """
<|endoftext|><s>
User:
Kan du berätta lite om Lund?
<s>
Bot:
""".strip()

# (Optional) - define a stopping criteria
# We ideally want the model to stop generate once the response from the Bot is generated
class StopOnTokenCriteria(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0, -1] == self.stop_token_id

stop_on_token_criteria = StopOnTokenCriteria(stop_token_id=tokenizer.bos_token_id)
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs=input_ids,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.6,
    top_p=1,
    stopping_criteria=StoppingCriteriaList([stop_on_token_criteria])
)[0]

generated_text = tokenizer.decode(generated_token_ids[len(input_ids[0]):-1])

In [ ]:
print(generated_text)